# **Support Vector Machines (SVM)**

In this Jupyter Notebook, we will build a "toy" SVM model using a mini dataset step by step.

Our goal is to run all the cells below one by one from top to bottom.

***

## **1. Install Dependencies**

**quadprog** is a Python package for solving quadratic programming problems. We can install it using the following command:
```
pip install quadprog
```
Note: Windows users may need to install Visual C++ 14 first (https://visualstudio.microsoft.com/visual-cpp-build-tools/).

Note: If using Python 3.11, there will be an inevitable error message. Please use Python 3.10 or older versions for this notebook. 

In [1]:
import quadprog
import numpy as np

# Helper function
def quadprog_solve_qp(P, q, G=None, h=None, A=None, b=None):
    qp_G = .5 * (P + P.T)   # make sure P is symmetric
    qp_a = -q
    if A is not None:
        qp_C = -np.vstack([A, G]).T
        qp_b = -np.hstack([b, h])
        meq = A.shape[0]
    else:   # no equality constraint
        qp_C = -G.T
        qp_b = -h
        meq = 0
    return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]

# Toy data
X = np.array([
    [0, 0],
    [2, 0],
    [0, 2],
    [3, 3],
    [4, 4]
])
Y = np.array([-1, -1, -1, 1, 1])

***

## **Goal**
We want to build an SVM model on the toy dataset: 

$
    x^{(1)} = (0,0),\ y^{(1)}=-1\\
    x^{(2)} = (2,0),\ y^{(2)}=-1\\
    x^{(3)} = (0,2),\ y^{(3)}=-1\\
    x^{(4)} = (3,3),\ y^{(4)}=1\\
    x^{(5)} = (4,4),\ y^{(5)}=1\\
$

We need to solve the quadratic programming (QP) problem as the following form:

$
    \min_{\alpha}\big( \frac{1}{2}\alpha^{T}Q\alpha - (\textbf{1})^{T}\alpha \big) \\
    \text{subject to: } y^{T}\alpha=0,\ \alpha\geq 0
$

The quadprog package by defaualt solves the QP as this form:

$
    \min_{x}\big( \frac{1}{2}x^{T}Px + q^{T}x \big) \\
    \text{subject to: } Gx\leq h,\ Ax = b
$

Therefore, in order to use quadprog, we need to establish the responding relationships between variables: 
$P=Q$, $q = -(\textbf{1})^{T}$, $G = -(\textbf{1})^{T}$, $h=(\textbf{0})^{T}$, $A=y^T$, $b=(\textbf{0})^{T}$

***

## **2. Compute Matrix $Q$**

First, we need to use $x^{(i)}$ and $y^{(i)}$ to compute matrix $Q$:

$
    Q = \begin{bmatrix}
    y^{(1)}y^{(1)}x^{(1)T}x^{(1)} & y^{(1)}y^{(2)}x^{(1)T}x^{(2)} & \dots & y^{(1)}y^{(5)}x^{(1)T}x^{(5)} \\
    y^{(2)}y^{(1)}x^{(2)T}x^{(1)} & y^{(2)}y^{(2)}x^{(2)T}x^{(2)} & \dots & y^{(2)}y^{(5)}x^{(2)T}x^{(5)} \\
    \vdots & \vdots & \ddots & \vdots \\
    y^{(5)}y^{(1)}x^{(5)T}x^{(1)} & y^{(5)}y^{(2)}x^{(5)T}x^{(2)} & \dots & y^{(5)}y^{(5)}x^{(5)T}x^{(5)} \\
    \end{bmatrix}
$


In [2]:
Q = np.zeros((5, 5))

for i in range(Q.shape[0]):
    for j in range(Q.shape[1]):
        # Use the ith and jth examples in X and Y to compute Q_ij
        # Hint: Q_ij = y^i * y^j * (x^i @ x^j)
        Q[i, j] = np.dot(np.dot(Y[i], Y[j]), np.dot(X[i], X[j]))

print('Q = ', Q)

Q =  [[ 0.  0.  0.  0.  0.]
 [ 0.  4.  0. -6. -8.]
 [ 0.  0.  4. -6. -8.]
 [ 0. -6. -6. 18. 24.]
 [ 0. -8. -8. 24. 32.]]


In [3]:
P = Q + np.eye(5)*1e-5 # To solve the non-positive finite issue

# Hint: Use np.ones(), q is of length 5
q =  -np.ones(5)

# Hint: G is a matrix whose diagnal elements are 1s, and other elements are 0s. Use np.eye()
G = -np.eye(5)

# Hint: h is of length 5, with all zeros; Use np.zeros()
h = np.zeros(5)

A = Y.reshape((1,5))

# Hint: b is of length 1, with zero value; Use np.zeros()
b = np.zeros(1)

print('q = ', q)
print('G = ', G)
print('h = ', h)
print('b = ', b)

q =  [-1. -1. -1. -1. -1.]
G =  [[-1. -0. -0. -0. -0.]
 [-0. -1. -0. -0. -0.]
 [-0. -0. -1. -0. -0.]
 [-0. -0. -0. -1. -0.]
 [-0. -0. -0. -0. -1.]]
h =  [0. 0. 0. 0. 0.]
b =  [0.]


In [4]:
# Hint: Call quadprog_solve_qp() with the correct arguments
solution = np.round(quadprog_solve_qp(P, q, G, h, A, b), 3)

print('solution = ', solution)
print('The support vectors are: ', X[solution > 0, ])

solution =  [0.    0.125 0.125 0.25  0.   ]
The support vectors are:  [[2 0]
 [0 2]
 [3 3]]


***

## **3. Solve the Decision Boundary**

We need to use the support vectors to solve the $w$ and $b$ in the decision boundary $w^Tx+b=0$, and use the property that a support vector $x^{(k)}$ must satistify $y^{(k)}(w^Tx^{(k)}+b) = 1$. Solving it with a paper and pen is possible by listing linear equations.

**NOTE**: We must solve this task on paper and only need to provide the answers for `w1`, `w2`, and `b`.

*Hint*: We must solve the following linear equations:

$\begin{cases} y^{(2)}(w^Tx^{(2)}+b) = 1 \\ y^{(3)}(w^Tx^{(3)}+b) = 1 \\ y^{(4)}(w^Tx^{(4)}+b) = 1\end{cases}$

In [5]:
# ANSWERS
w1 = 0.5
w2 = 0.5
b = -2

print('w1 = ', w1)
print('w2 = ', w2)
print('b = ', b)

w1 =  0.5
w2 =  0.5
b =  -2
